### 摘要
书接上文，上文《基于TensorFlow使用Softmax回归模型进行手写体识别》获得的准确率只有92%，比较低的准确率。本文引入CNN(卷积神经网络)来解决同样的手写体识别问题，准确率会有明显提升。本文分为3个部分，1.神经网络简介，2.更优的卷积神经网络，3.使用简单的CNN实现手写体识别。

### 神经网络简介
要想对神经网络有深刻的理解，可能需要从数学角度进行深入的解读，涉及空间变换、非线性引入、线性可分、梯度下降算法、反向传播算法等等。本文暂不展开，仅从宏观角度，有个大体的认识。来一张典型的人工神经网络图：
![nn](https://static.leiphone.com/uploads/new/article/740_740/201705/591023daa6090.png?imageMogr2/format/jpg/quality/90)
上图是一个3层的神经网络，包括输入层，隐藏层，输出层，前一层的输出作为后一层的输入。上图是一个全连接神经网络，若在中间只使用一层隐藏层，参数 w 就有 784×15=11760 多个，这很容易看出使用全连接神经网络处理图像中的需要训练参数过多的问题。所以，接下来引入卷积神经网络。  
上图中每一层中的一个圆圈代表一个特征值，比如输入层当前有8个圆圈，每个圆圈代表房屋面积、房间数，或每个圆圈代表手写体图像中的一个像素点。
### 卷积神经网络简介（Convolutional Neural Network,CNN）
CNN解决了全连接神经网络参数过多的问题，卷积层的神经元只与前一层的部分神经元节点相连，即它的神经元间的连接是非全连接的，且同一层中某些神经元之间的连接的权重 w 和偏移 b 是共享的（即相同的），这样大量地减少了需要训练参数的数量。  
在CNN的输入层中，（图片）数据输入的格式 与 全连接神经网络的输入格式（一维向量）不太一样。CNN的输入层的输入格式保留了图片本身的结构。  
对于黑白的 28×28 的图片，CNN的输入是一个 28×28 的的二维神经元，如下图所示：
![](https://static.leiphone.com/uploads/new/article/740_740/201705/591024439ff28.png?imageMogr2/format/jpg/quality/90)
卷积神经网络CNN的结构一般包含这几个层：
- 输入层：用于数据的输入
- 卷积层：使用卷积核进行特征提取和特征映射
- 激励层：由于卷积也是一种线性运算，因此需要增加非线性映射
- 池化层：进行下采样，对特征图稀疏处理，减少数据运算量。
- 全连接层：通常在CNN的尾部进行重新拟合，减少特征信息的损失
- 输出层：用于输出结果  

当然中间还可以使用一些其他的功能层:
- 归一化层（Batch Normalization）：在CNN中对特征的归一化
- 切分层：对某些（图片）数据的进行分区域的单独学习
- 融合层：对独立进行特征学习的分支进行融合


### 卷积层
CNN的隐藏层称为卷积层。在卷积层中有几个重要的概念：
  - local receptive fields（感受视野）
  - shared weights（共享权值）  
  
假设输入的是一个 28×28 的的二维神经元，我们定义5×5 的 一个 local receptive fields（感受视野），通过下图直观感受一下：
![](https://static.leiphone.com/uploads/new/article/740_740/201705/591024892abaa.png?imageMogr2/format/jpg/quality/90)
设移动的步长为1：从左到右扫描，每次移动 1 格，扫描完之后，再向下移动一格，再次从左到右扫描。具体过程见下面的动图：
![](https://static.leiphone.com/uploads/new/article/740_740/201705/591024b0dc6a7.gif)
一个感受视野带有一个卷积核，我们将 感受视野 中的权重 w 矩阵称为 卷积核 ；将感受视野对输入的扫描间隔称为步长（stride）；当步长比较大时（stride>1），为了扫描到边缘的一些特征，感受视野可能会“出界”，这时需要对边界扩充(pad)，边界扩充可以设为 0 或 其他值。步长 和 边界扩充值的大小由用户来定义。  
卷积核的大小由用户来定义，即定义的感受视野的大小；卷积核的权重矩阵的值，便是卷积神经网络的参数，为了有一个偏移项 ，卷积核可附带一个偏移项 b ，它们的初值可以随机来生成，可通过训练进行变化。  
上面动图所示的卷积核的权重矩阵就是：
$$
 \left[
 \begin{matrix}
   1 & 0 & 1 \\
   0 & 1 & 0 \\
   1 & 0 & 1
  \end{matrix}
  \right]
$$  
因此 感受视野 扫描时可以计算出下一层神经元的值为：
![](https://static.leiphone.com/uploads/new/article/740_740/201705/59102506ee8c0.png?imageMogr2/format/jpg/quality/90)



### 激励函数
激励层主要对卷积层的输出进行一个非线性映射，因为卷积层的计算还是一种线性计算。使用的激励函数一般为ReLu函数：f(x)=max(x,0)

### 池化层
当输入经过卷积层时，若感受视野比较小，步长stride比较小，得到的feature map （特征图）还是比较大，可以通过池化层来对每一个feature map进行降维操作，输出的深度还是不变的，依然为 feature map 的个数。   
一般池化的计算方法有两种：
- Max pooling：取“池化视野”矩阵中的最大值
- Average pooling：取“池化视野”矩阵中的平均值  

下图是一个池化窗口是2x2，步长为2的池化过程，使用Max pooling方法：  
![](https://static.leiphone.com/uploads/new/article/740_740/201705/591025f062f4f.png?imageMogr2/format/jpg/quality/90)
最后可将3个24×24的 feature map 下采样(即池化)得到 3 个 24×24 的特征矩阵：
![](https://static.leiphone.com/uploads/new/article/740_740/201705/5910260b1d14c.png?imageMogr2/format/jpg/quality/90)
这里说一下通道的概念，输入层是图片的话，黑白图片只有一种颜色，也就是有一个通道。彩色图片(RGB图)具有3个颜色，则他作为输入层的话，就有3个通道。
卷积层的通道可以自定义，一个卷积核的权重矩阵就会形成一个通道。

### 基于tensorflow构建CNN进行手写体识别
网络结构如下图所示：
![](https://tensorflow.google.cn/images/mnist_deep.png)
该网络有2个卷积层、2个池化层、2个全连接层。
### 加载数据集
数据集和上一篇文章《基于TensorFlow使用Softmax回归模型进行手写体识别》一致。
训练数据集大小是55000，验证数据集大小是10000.

In [27]:
import os
from tensorflow.examples.tutorials.mnist import input_data

data_path = '%s%smnist_dataset' % (os.getcwd(), os.sep)
# one_hot变量是指一个特殊的向量，例如，3表示为[0,0,0,1,0,0,0,0,0,0]
mnist = input_data.read_data_sets(data_path, one_hot=True)

Extracting E:\work\code\AI\deeplearning\mnist_dataset\train-images-idx3-ubyte.gz
Extracting E:\work\code\AI\deeplearning\mnist_dataset\train-labels-idx1-ubyte.gz
Extracting E:\work\code\AI\deeplearning\mnist_dataset\t10k-images-idx3-ubyte.gz
Extracting E:\work\code\AI\deeplearning\mnist_dataset\t10k-labels-idx1-ubyte.gz


### 定义权重和偏移量函数

In [28]:
import tensorflow as tf
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

权重的初始值使用了符合标准差是0.1的正态分布的数值，形成形状为shape的矩阵。
偏移量的初始值是0.1。
### 定义卷积层和池化层
- tf.nn.conv2d说明  
该方法用于生成一个卷积层。W是权重矩阵，它作为参数后面代码提供，strides为步长，4个值分别为[batch , in_height , in_width, in_channels]这4个维度的移动步长，第1，4个值都是1，代表每次一张图片的1个通道，第2，3个值则是卷积核在水平和垂直方向移动的步长。padding='SAME'表明边界使用0填充，使得输出矩阵和输入矩阵的大小保持一致。
- tf.nn.max_pool说明  
该方法用于生成一个池化层。value：表示池化的输入：一个4维格式的数据，数据的shape由data_format决定，默认情况下shape为[batch, height, width, channels]。ksize：表示池化窗口的大小：一个长度为4的一维列表，一般为[1, height, width, 1]，因不想在batch和channels上做池化，则将其值设为1。其他参数含义同卷积层。


In [29]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


### 第一个卷积层
它包括一个卷积层，紧跟一个池化层。这个卷积层将使用5x5的卷积核计算出32个特征(也叫通道)。它的权重张量的形状是[5,5,1,32]。前两维是卷积核大小，下一个是输入通道数，最后一个是输出通道数。相应地偏移量是一个32维的向量，与输出通道数一致。
这一层的参数个数是5x5x32=800

In [30]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

为了对接这个卷积层，输入数据必须按照4维张量[-1, 28, 28, 1]的形状进行转换，第1维是图片数量，可变。2，3维是图片的宽和高，最后一维是颜色通道的数量。  

In [31]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])

接下来，先根据输入x_image和权重进行卷积计算，再加上偏移量，对其结果再进行relu激活函数操作。这样得到卷积层的结果。
紧接着，对上述输出结果进行池化，得到池化结果。

In [32]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### 第二个卷积层
为了构建一个深度神经网络，我们构造多个此类型的层，第二层是一个64个特征，卷积核是5x5。

In [33]:
#上一层有32个特征，此处输入通道就是32了
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### 全连接层
至此，图片的大小已经被缩减至7x7，现在我们增加一个带有1024个神经元的全连接层，以便获取上一层整张图的特征，而不像卷积层那样获取局部特征。我们把池化层的张量(形如[7,7,64])转换成一组向量，然后乘以权重矩阵，加上偏移量，再应用激活函数ReLU。

In [34]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


### Dropout
要减少过拟合，我们需要在输出层之前应用Dropout。在Dropout过程中，我们需要定义一个占位符代表一个神经元的输出值被保留的概率。这样，我们就可以在训练时打开Dropout，在预测时关闭Dropout。
关于Dropout的一点额外说明，对于上面这样的小型网络，有没有Dropout对于性能几乎是一样的。Dropout在防止过拟合方面是非常高效的，但，它在大型神经网络的训练时是最有用的。

In [35]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### 输出层
最后，我们增加一层，就像我们上一篇文章讲到的Softmax回归一样，用来做分类。

In [36]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

## 训练和评估模型
我们的模型效果如何呢？我们要对上面定义的卷积神经网络模型进行训练和评估。这部分代码和上一篇文章使用Softmax回归基本一致。
些许不同，如下：
- 我们使用更复杂的ADAM优化器替换掉了陡峭的梯度下降优化器。
- 我增加了额外的参数keep_prob用来指定Dropout rate。
- 在训练过程中，我们每100次迭代进行一次日志输出。  

该训练过程进行了200轮迭代，在我的工作笔记本上耗时～22分钟，在10000个测试集(mnist.test.images)上验证得到准确率是0.9015。提醒，这个在我笔记本上跑，短暂性死机了。
在Titan X GPU上跑，2000轮迭代，耗时12s，测试集上准确率是0.9749。20000轮迭代，耗时90s，准确率是0.992。40000轮迭代，准确率是0.9928，没有明显提升。

In [37]:
import datetime

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(200):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      time_stamp = datetime.datetime.now()
      print ("time_stamp:%s" % time_stamp.strftime('%Y.%m.%d-%H:%M:%S'))
      
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  time_stamp = datetime.datetime.now()
  print ("time_stamp:%s" % time_stamp.strftime('%Y.%m.%d-%H:%M:%S'))
  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
  time_stamp = datetime.datetime.now()
  print ("time_stamp:%s" % time_stamp.strftime('%Y.%m.%d-%H:%M:%S'))

time_stamp:2017.11.28-09:08:44
step 0, training accuracy 0.12
time_stamp:2017.11.28-09:09:13
step 100, training accuracy 0.76
time_stamp:2017.11.28-09:09:41
test accuracy 0.9015
time_stamp:2017.11.28-09:30:12


### 参考文章
1. [从理论到实践，手把手教你如何用 TensorFlow 实现 CNN](https://www.leiphone.com/news/201705/HH3BbIfCqAtOAMbu.html), 说明该文章的示例代码有问题，数据集使用不正确，第一个池化层没有使用。
2. [Deep MNIST for Experts](https://tensorflow.google.cn/get_started/mnist/pros#a1)